## Import extreme multi-label dataset
- in extreme-sparse dataset, test wide n deep 

In [27]:
import os

import pandas as pd
import numpy as np
from scipy.io import loadmat

In [28]:
dataset="bibtex"
data_dir='data/{}'.format(dataset)
print(data_dir)
def load_input():
    data = list(loadmat(data_dir + '/input.mat')['data'][0][0])
    return data[:4]

data/bibtex


In [29]:
train_X, train_Y, test_X, test_Y = load_input()

In [30]:
train_X = train_X.toarray()
train_Y = train_Y.toarray()
test_X = test_X.toarray()
test_Y = test_Y.toarray()

print(train_X.shape)
print(train_Y.shape)

(4880, 1836)
(4880, 159)


In [34]:
pd.DataFrame(train_X).head()

,0,1,2,3,4,5,6,7,8,9,...,1826,1827,1828,1829,1830,1831,1832,1833,1834,1835
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Coding wide-n-deep with keras

In [35]:
from scipy.io import loadmat

from keras.models import Sequential, Model
from keras.layers import Input, Dense, concatenate
from keras.optimizers import Adam


# dataset
dataset = "bibtex"
data_dir = 'data/{}'.format(dataset)

def load_input():
    data = list(loadmat(data_dir + '/input.mat')['data'][0][0])
    return data[:4]


class Deep:
    def __init__(self, batch_size, epochs, learning_rate, input_dim, output_dim):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.model = self.classifier()

    def classifier(self):
        model = Sequential()
        model.add(Dense(100, activation='relu', input_dim=self.input_dim))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(self.output_dim, activation='softmax'))

        optimizer = Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999)
        model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
        return model

    def fit(self, x, y):
        self.model.fit(x, y, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.2)

    def print_performance(self, x, y):
        performance_test = self.model.evaluate(x, y, batch_size=self.batch_size)
        print('Test Loss and Accuracy ->', performance_test)


class Wide:
    def __init__(self, batch_size, epochs, learning_rate, input_dim, output_dim):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.model = self.classifier()

    def classifier(self):
        model = Sequential()
        model.add(Dense(self.output_dim, activation='softmax', input_dim=self.input_dim))

        optimizer = Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999)
        model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
        return model

    def fit(self, x, y):
        self.model.fit(x, y, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.2)

    def print_performance(self, x, y):
        performance_test = self.model.evaluate(x, y, batch_size=self.batch_size)
        print('Test Loss and Accuracy ->', performance_test)


class WideAndDeep:
    def __init__(self, batch_size, epochs, learning_rate, input_dim, output_dim):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.model = self.classifier()

    def classifier(self):
        optimizer = Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999)

        # wide part
        wide = Input(shape=(self.input_dim,))

        # deep part
        deep_input = Input(shape=(self.input_dim,))
        deep = Dense(100, activation='relu')(deep_input)
        deep = Dense(50, activation='relu')(deep)

        # concatenate : wide and deep
        wide_n_deep = concatenate([wide, deep])
        wide_n_deep = Dense(self.output_dim, activation='softmax')(wide_n_deep)
        model = Model(inputs=[wide, deep_input], outputs=wide_n_deep)
        model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
        return model

    def fit(self, wide_x, deep_x, y):
        self.model.fit([wide_x, deep_x], y, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.2)

    def print_performance(self, wide_x, deep_x, y):
        performance_test = self.model.evaluate([wide_x, deep_x], y, batch_size=self.batch_size)
        print('Test Loss and Accuracy ->', performance_test)


def main(model_param):
    # prepare dataset
    x_train, y_train, x_test, y_test = load_input()
    x_train = x_train.toarray()
    y_train = y_train.toarray()
    x_test = x_test.toarray()
    y_test = y_test.toarray()

    # prepare hyper parameter
    batch_size = 500
    epochs = 30
    learning_rate = 0.001
    input_dim = x_train.shape[1]
    output_dim = y_train.shape[1]
    

    if model_param == "deep":
        deep = Deep(batch_size, epochs, learning_rate, input_dim, output_dim)
        deep.fit(x_train, y_train)
        deep.print_performance(x_test, y_test)
    elif model_param == 'wide':
        wide = Wide(batch_size, epochs, learning_rate, input_dim, output_dim)
        wide.fit(x_train, y_train)
        wide.print_performance(x_test, y_test)
    else:
        wide_n_deep = WideAndDeep(batch_size, epochs, learning_rate, input_dim, output_dim)
        wide_n_deep.fit(x_train, x_train, y_train)
        wide_n_deep.print_performance(x_test, x_test, y_test)

        # prediction for individual and y_column rank
        x_predict_test = x_test[np.newaxis, 0, :]
        y_predict_test = y_test[0]
        result = wide_n_deep.model.predict([x_predict_test, x_predict_test])
        print('result predicted:', result)
        print('result real:', y_predict_test)

        # select top 10 y's column index in result(softmax prediction)
        top_10_y_column = result[0].argsort()[-10:][::-1].tolist()
        print('result top 10:', top_10_y_column)


if __name__ == '__main__':
    main('widendeep')

Train on 3904 samples, validate on 976 samples
Epoch 1/30
3904/3904 [==============================] - 1s 164us/step - loss: 11.8599 - acc: 0.0236 - val_loss: 11.8551 - val_acc: 0.0625
Epoch 2/30
3904/3904 [==============================] - 0s 117us/step - loss: 10.9283 - acc: 0.0774 - val_loss: 11.3183 - val_acc: 0.0840
Epoch 3/30
3904/3904 [==============================] - 0s 112us/step - loss: 10.2314 - acc: 0.1283 - val_loss: 10.8678 - val_acc: 0.1650
Epoch 4/30
3904/3904 [==============================] - 0s 114us/step - loss: 9.5705 - acc: 0.2059 - val_loss: 10.3990 - val_acc: 0.1834
Epoch 5/30
3904/3904 [==============================] - 0s 116us/step - loss: 8.9016 - acc: 0.2403 - val_loss: 9.9461 - val_acc: 0.1906
Epoch 6/30
3904/3904 [==============================] - 0s 114us/step - loss: 8.2707 - acc: 0.2713 - val_loss: 9.5387 - val_acc: 0.2039
Epoch 7/30
3904/3904 [==============================] - 0s 116us/step - loss: 7.6986 - acc: 0.3186 - val_loss: 9.1815 - val_acc: 0